In [53]:
import numpy as np
import pandas as pd
import statsmodels.formula.api as sm
import statsmodels.tsa.stattools as ts
import statsmodels.tsa.vector_ar.vecm as vm

def get_cointegrated_pairs(dataframe, critical_level = 0.01, train_test_ratio = 0.6):
    
#     dataframe.set_index('Date', inplace=True)
    
    # train set / test set split
    train_df = dataframe[:int(len(dataframe.index)*train_test_ratio)]
    test_df = dataframe[int(len(dataframe.index)*train_test_ratio):]
    
    # Create matrix
    n = train_df.shape[1] # number of etfs
    pvalue_matrix = np.ones((n,n))
    keys = train_df.keys()
    pairs = []
    
    for i in range(n):
        for j in range(i+1, n):
            pair_1 = train_df[keys[i]]
            pair_2 = train_df[keys[j]] # obtain the price of two contract
            
            result = ts.coint(pair_1, pair_2) # get conintegration
            pvalue = result[1] # get the pvalue
            pvalue_matrix[i, j] = pvalue
            
            if pvalue < critical_level: # if p-value less than the critical level
                pairs.append((keys[i], keys[j], pvalue)) # record the contract with that p-value
        
        if i%10==0:
            print("{}th calculation completed".format(i))
            
    return pvalue_matrix, pairs

In [51]:
etfs = pd.read_csv('ETFs_merged')
etfs['Date'] = pd.to_datetime(etfs['Date'])
etfs.set_index('Date', inplace = True)
etfs.drop(['Unnamed: 0'], axis = 1, inplace=True)
etfs.head()

,KODEX 200,TIGER KRX100,TIGER 반도체,TIGER 은행,KODEX 자동차,TREX 중소형가치,TIGER 방송통신,KODEX China H,KODEX 일본TOPIX100,KODEX 삼성그룹,...,KINDEX 중국본토CSI300,TIGER 구리실물,TIGER 베타플러스,KODEX 중국본토 A50,KBSTAR 중국본토대형주CSI100,TIGER 로우볼,KBSTAR 주식혼합,KBSTAR 채권혼합,KODEX 미국S&P바이오(합성),ARIRANG 글로벌MSCI(합성 H)
Date,,,,,,,,,,,,,,,,,,,,,
2014-01-13,25630.0,42415.0,17725.0,8250.0,21305.0,7110.0,9240.0,15390.0,9725.0,5955.0,...,11450.0,7910.0,11480.0,8230.0,7855.0,11840.0,29325.0,49705.0,12230.0,10255.0
2014-01-14,25595.0,42445.0,17630.0,8115.0,21350.0,7070.0,9215.0,15455.0,9580.0,5945.0,...,11460.0,7920.0,11460.0,8285.0,7925.0,11730.0,29425.0,49785.0,12005.0,10180.0
2014-01-15,25670.0,42480.0,17450.0,8165.0,21440.0,7080.0,9225.0,15620.0,9640.0,5990.0,...,11505.0,7965.0,11450.0,8315.0,7895.0,11755.0,29390.0,49735.0,12135.0,10230.0
2014-01-16,25790.0,42705.0,17605.0,8160.0,21370.0,7110.0,9195.0,15615.0,9660.0,5990.0,...,11555.0,7965.0,11525.0,8375.0,7985.0,11810.0,29350.0,49780.0,12195.0,10280.0
2014-01-17,25550.0,42380.0,17635.0,8055.0,21200.0,7095.0,9150.0,15550.0,9635.0,5965.0,...,11390.0,7970.0,11435.0,8195.0,7820.0,11780.0,29310.0,49695.0,12400.0,10275.0


In [52]:
etfs.info()

<class 'pandas.core.frame.DataFrame'>
DatetimeIndex: 1270 entries, 2014-01-13 to 2019-03-19
Data columns (total 70 columns):
KODEX 200                 1270 non-null float64
TIGER KRX100              1270 non-null float64
TIGER 반도체                 1270 non-null float64
TIGER 은행                  1270 non-null float64
KODEX 자동차                 1270 non-null float64
TREX 중소형가치                1270 non-null float64
TIGER 방송통신                1270 non-null float64
KODEX China H             1270 non-null float64
KODEX 일본TOPIX100          1270 non-null float64
KODEX 삼성그룹                1270 non-null float64
KODEX 기계장비                1270 non-null float64
KODEX 증권                  1270 non-null float64
KOSEF 블루칩                 1270 non-null float64
KOSEF 고배당                 1270 non-null float64
TIGER 라틴35                1270 non-null float64
KBSTAR 5대그룹주              1270 non-null float64
KINDEX 삼성그룹섹터가중           1270 non-null float64
KODEX 에너지화학               1270 non-null float64
TIGER 차이나항셍

In [55]:
pavlue, pairs = get_cointegrated_pairs(etfs)

0th calculation completed
10th calculation completed
20th calculation completed
30th calculation completed
40th calculation completed
50th calculation completed
60th calculation completed


In [65]:
pairs

[('KODEX 자동차', 'KODEX 일본TOPIX100', 0.00535690271772077),
 ('TIGER 방송통신', 'KODEX 콩선물(H)', 0.00328646106974724),
 ('KODEX 삼성그룹', 'KODEX 구리선물(H)', 0.007496495588200022),
 ('KODEX 삼성그룹', 'TIGER 헬스케어', 0.008940145066585978),
 ('KOSEF 블루칩', 'TIGER 농산물선물Enhanced(H)', 0.006785465924616681),
 ('KOSEF 고배당', 'TIGER 미국S&P500선물(H)', 0.004069103862396616),
 ('KINDEX 삼성그룹섹터가중', 'KODEX 구리선물(H)', 0.004508978332751525),
 ('KODEX 골드선물(H)', 'TIGER 금은선물(H)', 0.002440944434781736),
 ('TIGER 미국나스닥100', 'TIGER 200 경기소비재', 0.0007374295661598058),
 ('KOSEF 미국달러선물', 'KBSTAR 수출주', 0.008352784948765403),
 ('TIGER 삼성그룹펀더멘털', 'KODEX 구리선물(H)', 0.006317554426175247),
 ('TIGER 삼성그룹펀더멘털', 'KBSTAR 우량업종', 0.004509795662494546),
 ('KODEX 구리선물(H)', 'KBSTAR 우량업종', 0.002264362295516075),
 ('TIGER 금속선물(H)', 'KBSTAR 우량업종', 0.0017616599003398336),
 ('KBSTAR 우량업종', 'TIGER 구리실물', 0.0026049486423122756),
 ('KODEX 보험', 'KODEX 은선물(H)', 0.005944279074312987),
 ('파워 코스피100', 'KODEX MSCI Korea', 0.0006227483291739576)]